<a href="https://colab.research.google.com/github/hamidi311/Population-Collapse-Analysis/blob/main/Collapse_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Load data
url = "https://raw.githubusercontent.com/hamidi311/Population-Collapse-Analysis/main/ML_model/predi_model_data%20(1).csv"
df = pd.read_csv(url)

# Filter for recent years
df = df[df['Year'] >= 2010]

# Select key columns and drop missing values
df = df[['Fertility Rate', 'Population Over 65', 'Birth Rate']].dropna().copy()

# Use temporary logic only to generate labels for training
def label_status(row):
    if row['Fertility Rate'] < 2.1 and row['Population Over 65'] > 15:
        return 'Collapsing'
    elif row['Fertility Rate'] < 2.1 or row['Population Over 65'] > 15:
        return 'At Risk of collapsing'
    else:
        return 'Growing'

df['status'] = df.apply(label_status, axis=1)

# Encode labels
le = LabelEncoder()
df['status_encoded'] = le.fit_transform(df['status'])

# Define features (X) and labels (y)
X = df[['Fertility Rate', 'Population Over 65']]
y = df['status_encoded']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print("Evaluation Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Predict using ML only (no logic)
def predict_status(fertility_rate, pop_over_65):
    input_df = pd.DataFrame([[fertility_rate, pop_over_65]],
                            columns=['Fertility Rate', 'Population Over 65'])
    prediction_encoded = model.predict(input_df)[0]
    prediction_label = le.inverse_transform([prediction_encoded])[0]
    return prediction_label

# Interactive CLI Prediction
def interactive_prediction():
    print("\n📍 Population Collapse Risk Predictor (ML-powered)")
    country = input("Enter Country Name: ")
    fertility_rate = float(input("Enter Fertility Rate: "))
    pop_over_65 = float(input("Enter Population Over 65 (%): "))

    prediction = predict_status(fertility_rate, pop_over_65)
    print(f"\n🔍 Prediction for {country}: {prediction}")

# Run interactive prediction
interactive_prediction()


Evaluation Report:
                       precision    recall  f1-score   support

At Risk of collapsing       1.00      1.00      1.00       187
           Collapsing       1.00      1.00      1.00       134
              Growing       1.00      1.00      1.00       351

             accuracy                           1.00       672
            macro avg       1.00      1.00      1.00       672
         weighted avg       1.00      1.00      1.00       672


📍 Population Collapse Risk Predictor (ML-powered)
Enter Country Name: algeria 
Enter Fertility Rate: 2
Enter Population Over 65 (%): 2

🔍 Prediction for algeria : At Risk of collapsing
